In [1]:
import random
import time
from rangeindex import RangeIndex
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
alpha = 'qwertyuiopasdfghjklzxcvbnm'
nums = list(range(10**4))
w_length = 5

class Thing():
    def __init__(self):
        self.f0 = random.random()
        self.f1 = random.random()
        self.f2 = random.random()
        self.f3 = random.random()
        self.f4 = random.random()
        self.f5 = random.random()
        self.f6 = random.random()
        self.f7 = random.random()
        self.f8 = random.random()
        self.f9 = random.random()
        self.data = '12894398ur8fehsiungjksgkjshgkdkh'
"""
self.c0 = random.choice(alpha)
self.c1 = random.choice(alpha)
self.c2 = random.choice(alpha)
self.c3 = random.choice(alpha)
self.c4 = random.choice(alpha)
self.c5 = random.choice(alpha)
self.c6 = random.choice(alpha)
self.c7 = random.choice(alpha)
self.c8 = random.choice(alpha)
self.c9 = random.choice(alpha)
self.w0 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w1 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w2 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w3 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w4 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w5 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w6 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w7 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w8 = ''.join(random.choice(alpha) for _ in range(w_length))
self.w9 = ''.join(random.choice(alpha) for _ in range(w_length))
self.i0 = random.choice(nums)
self.i1 = random.choice(nums)
self.i2 = random.choice(nums)
self.i3 = random.choice(nums)
self.i4 = random.choice(nums)
self.i5 = random.choice(nums)
self.i6 = random.choice(nums)
self.i7 = random.choice(nums)
self.i8 = random.choice(nums)
self.i9 = random.choice(nums)
"""
EXP = 7
things = [Thing() for _ in range(10**EXP)]

Let's benchmark on some floats!

In [3]:
engines = ['pandas', 'sqlite']

In [4]:
def generate_float_range(n_results=1, n_idx=1, dataset_size=10**EXP) -> tuple[float, float]:
    """
    Generate a range of float values that, when queried, will return approx n_results items.
    Example: if n_idx is 1, and we want 1/10 of the items in the dataset, we could return (0.5, 0.6)
    as that will be 1/10 of the items.
    But if n_idx is 2, we want a range of 0.1**(1/2) items, to account for the fact that we have 
    two independent value ranges.
    """
    width = (n_results / dataset_size)**(1/n_idx)
    lim_1 = random.random()*(1-width)
    lim_2 = lim_1 + width
    return lim_1, lim_2


In [5]:
def bench_floats(n_idx=1, n_runs=5):
    results = {}
    n_runs = 10
    fields = {f'f{i}': float for i in range(n_idx)}
    ris = []
    for engine in engines:
        if engine == 'sqlite':
            ri = RangeIndex(things, fields,  engine, table_index=[('f0', 'f1')])
        else:
            ri = RangeIndex(things, fields,  engine)
        ris.append(ri)
    ris.append('linear')
    for ri in ris:
        for n_items in [10**e for e in range(0, EXP+1)]:
            # Run 2 bechmarks, take the second. Queries are randomized so this isn't a cache cheat.
            # Rather, this gives a "warm-up" round so that the data more accurately reflects
            # the performance you'd get at steady-state.
            # Without this, you get really weird distorted data.
            for _ in range(2):  
                t_tot = 0
                n_tot = 0
                for run in range(n_runs):
                    lo, hi = generate_float_range(n_items, n_idx)
                    if ri == 'linear':
                        q = ' and '.join([
                            f't.f{i} > {lo} and t.f{i} <= {hi}'
                            for i in range(n_idx)
                        ])
                        t0 = time.time()
                        items_got = eval(f'[t for t in things if {q}]')
                        t1 = time.time()
                    else:
                        q = ' and '.join([
                            f'f{i} > {lo} and f{i} <= {hi}'
                            for i in range(n_idx)
                        ])
                        t0 = time.time()
                        items_got = ri.find(q)
                        t1 = time.time()
                    t_tot += (t1-t0) / n_runs
                    n_tot += len(items_got) / n_runs
                if ri == 'linear':
                    cond = 'linear'
                else:
                    cond = ri.engine
                results[(cond, n_items)] = {'got': n_tot, 'time': t_tot}

    return results

In [6]:
def time_str(t: float) -> str:
    if t < 10 ** -4:
        return "{}μs".format(round(t * 10 ** 6, 1))
    elif t < 10 ** -1:
        return "{}ms".format(round(t * 10 ** 3, 1))
    else:
        return "{}s".format(round(t, 2))

def num_str(n: int) -> str:
    if n >= 10 ** 9:
        return "{}B".format(int(round(n / 10 ** 9)))
    elif n >= 10 ** 6:
        return "{}M".format(int(round(n / 10 ** 6)))
    elif n >= 10 ** 3:
        return "{}K".format(int(round(n / 10 ** 3)))
    else:
        return str(n)

In [7]:
# raw numbers, much better for tweaking

for n_idx in [2]:
    results = bench_floats(n_idx=n_idx)
    plt.figure(figsize=(20,10))
    if n_idx == 1:
        title = '1 field indexed'
    else:
        title = f'{n_idx} fields indexed'
    print(title)

    for i, engine in enumerate(engines + ['linear']):
        for n_items in [10**e for e in range(0, EXP+1)]:
            baseline = results[('linear', n_items)]['time']
            result = results[(engine, n_items)]['time']
            speedup = round(baseline/result, 3)
            print(f'{engine}, {num_str(n_items)}: {time_str(result)}\t\t({speedup})')


2 fields indexed
pandas, 1: 41.9ms		(24.603)
pandas, 10: 42.5ms		(25.807)
pandas, 100: 40.7ms		(23.818)
pandas, 1K: 43.4ms		(26.096)
pandas, 10K: 51.4ms		(22.533)
pandas, 100K: 96.8ms		(12.369)
pandas, 1M: 0.2s		(8.323)
pandas, 10M: 0.5s		(4.74)
sqlite, 1: 0.3ms		(3713.537)
sqlite, 10: 0.7ms		(1657.108)
sqlite, 100: 2.0ms		(495.059)
sqlite, 1K: 6.3ms		(181.248)
sqlite, 10K: 27.3ms		(42.444)
sqlite, 100K: 0.68s		(1.766)
sqlite, 1M: 1.43s		(1.147)
sqlite, 10M: 7.18s		(0.33)
linear, 1: 1.03s		(1.0)
linear, 10: 1.1s		(1.0)
linear, 100: 0.97s		(1.0)
linear, 1K: 1.13s		(1.0)
linear, 10K: 1.16s		(1.0)
linear, 100K: 1.2s		(1.0)
linear, 1M: 1.64s		(1.0)
linear, 10M: 2.37s		(1.0)


<Figure size 1440x720 with 0 Axes>

In [ ]:
matplotlib.rc('font', size=20)

# for n_idx in [1, 2, 5, 10]:
for n_idx in [2]:
    results = bench_floats(n_idx=n_idx)
    plt.figure(figsize=(20,10))
    if n_idx == 1:
        title = '1 field indexed'
    else:
        title = f'{n_idx} fields indexed'
    print(title)
    # plt.title(f'{title}', fontsize=30)

    for i, engine in enumerate(engines):
        x = []
        y = []
        for n_items in [10**e for e in range(0, EXP+1)]:
            baseline = results[('linear', n_items)]['time']
            result = results[(engine, n_items)]['time']
            speedup = round(baseline/result, 3)
            #print(f'{engine}, {n_items}: {speedup}x')
            x.append(n_items)
            y.append(speedup)
        plt.loglog(x, y, label=engine)
        plt.xlabel('number of items returned')
        plt.ylabel('relative speedup')
    plt.loglog([1, 10**EXP], [1, 1], '--', label='baseline')
    plt.xlabel('number of items returned', fontsize=30)
    plt.ylabel('relative speedup', fontsize=30)
    plt.legend()
    plt.show()

In [ ]:
# quick check, how much of that is dict lookup?
obj_ids = [id(t) for t in things]
obj_dict = {id(t): t for t in things}

t0 = time.time()
zz = [obj_dict[obj_id] for obj_id in obj_ids[:len(obj_ids)//100]]
t1 = time.time()
print('dict lookup of 100K items:', t1-t0)

t0 = time.time()
zz = [obj_dict[obj_id] for obj_id in obj_ids[:len(obj_ids)//10]]
t1 = time.time()
print('dict lookup of 1M items:', t1-t0)

t0 = time.time()
zz = [obj_dict[obj_id] for obj_id in obj_ids]
t1 = time.time()
print('dict lookup of 10M items:', t1-t0)

In [ ]:
(0.408)/(0.408-0.1179*5/6)